In [1]:
import pandas as pd
df = pd.DataFrame(dict(v1 = ["aaa", "abc", "ccc"],
                       v2 = ["2022-01", "2022-02", "2023"],
                       col = [1, 2, 3]))
df

,v1,v2,col
0,aaa,2022-01,1
1,abc,2022-02,2
2,ccc,2023,3


문자열로 구성된 Pandas Series객체는 .str 접근자를 사용할 수 있으며 해당 접근자는 다양한 문자열 처리 메서드를 제공함.

In [2]:
df["v1"].str.contains("a") # 문자열에 a가 포함 -> True

0     True
1     True
2    False
Name: v1, dtype: bool

In [3]:
df.loc[df["v1"].str.contains("a"), ]

,v1,v2,col
0,aaa,2022-01,1
1,abc,2022-02,2


필터링 전과 후가 변함이 없거나 필터링 후 남는 행의 개수가 0일 때 경고 또는 에러가 발생하지 않으니 주의

In [4]:
df.loc[df["v1"] == "a", ] # 에러 또는 경고가 발생하지 않음

,v1,v2,col


In [5]:
df.loc[df["v1"] == "aaa", ]

,v1,v2,col
0,aaa,2022-01,1


In [6]:
df["v2"].str.replace(pat = "-", # pattern
                     repl = "/") # replacement

0    2022/01
1    2022/02
2       2023
Name: v2, dtype: object

In [7]:
df["v2"] = df["v2"].str.replace(pat = "-", repl = "/")
df

,v1,v2,col
0,aaa,2022/01,1
1,abc,2022/02,2
2,ccc,2023,3


In [8]:
aa = "2022/01"
aa.split("/")

['2022', '01']

In [9]:
df["v2"].str.split("/")

0    [2022, 01]
1    [2022, 02]
2        [2023]
Name: v2, dtype: object

In [10]:
df["v2"].str.split("/").reset_index()

,index,v2
0,0,"[2022, 01]"
1,1,"[2022, 02]"
2,2,[2023]


In [11]:
df["v2"].str.split("/", expand = True)

,0,1
0,2022,01
1,2022,02
2,2023,None


In [12]:
df["v2"].str.split("/", expand = True).fillna("0")

,0,1
0,2022,01
1,2022,02
2,2023,0


In [13]:
df["v2"].str.split("/", expand = True).fillna("")

,0,1
0,2022,01
1,2022,02
2,2023,


In [14]:
df["v2"] + "/01"

0    2022/01/01
1    2022/02/01
2       2023/01
Name: v2, dtype: object

In [15]:
df["v2"].str.len() # 문자열 길이

0    7
1    7
2    4
Name: v2, dtype: int64

In [16]:
df

,v1,v2,col
0,aaa,2022/01,1
1,abc,2022/02,2
2,ccc,2023,3


In [17]:
df["v3"] = df["v1"].str.upper() # 대문자로~
df["v4"] = df["v3"].str.lower() # 소문자로~

In [18]:
df["v1"].str.count(pat = "a") # 특정 문자열 또는 패턴 등장 횟수

0    3
1    1
2    0
Name: v1, dtype: int64

In [19]:
# df["col"].str.zfill(2) # Error!!!
df["col"].astype("str").str.zfill(2) # 지정한 자릿수 보다 모자라면 0으로 채움

0    01
1    02
2    03
Name: col, dtype: object

In [20]:
df["v2"].str.slice(0, 4)

0    2022
1    2022
2    2023
Name: v2, dtype: object

### 정규표현식의 활용
텍스트를 핸들링 하기 위한 고급 문법으로 각 프로그래밍 언어마다 용법이 매우 유사하다. .str접근자의 메서드에 "pat"라는 인자가 있는 경우가 있는데 이는 pattern의 줄임말이며 정규표현식(regular expression)을 사용할 수 있음을 시사한다. 그리고 최근 Pandas는 .str접근자의 메서드에서 정규표현식을 사용하는 경우 "regex" 인자를 True로 설정하고 사용할 것을 권고하고 있다(Future Warning).  
※ Future Warning: 현재는 정상적으로 사용 가능하나 근 시일내 업데이트 되는 라이브러리의 버전에서 오동작 할 가능성이 매우 높은 사항에 대해 경고하는 것.

In [21]:
# !pip install pandas --user --upgrade

In [22]:
pd.__version__

'1.5.1'

In [23]:
df["v1"].str.contains("^a", regex = True) # a로 시작하는.

0     True
1     True
2    False
Name: v1, dtype: bool

In [24]:
df["v2"]

0    2022/01
1    2022/02
2       2023
Name: v2, dtype: object

In [34]:
# .extract() 메서드는 추출하고자 하는 문자열 패턴 좌우로 소괄호를 감싼다.
df["v2"].str.extract("^(.*?)$") # 모든 문자열, 개수 0~무한

,0
0,2022/01
1,2022/02
2,2023


In [27]:
# 중괄호는 직전의 패턴이 몇 번 반복되는지 지정한다.
df["v2"].str.extract("(.{4})") # 모든 문자열, 개수 4

,0
0,2022
1,2022
2,2023


In [28]:
df["v2"].str.extract("([0-9]{4})") # 숫자, 4개

,0
0,2022
1,2022
2,2023


In [29]:
# 소괄호 밖에 있는 문자인 슬래시(/)는 추출이 되지 않는 것을 확인 할 수 있다.
df["v2"].str.extract("([0-9]{4})/([0-9]{2})") # 숫자 4개 / 숫자 2개

,0,1
0,2022,01
1,2022,02
2,NaN,NaN


In [35]:
df["v2"].str.extract("([0-9]{4})(|/([0-9]{2}))")

,0,1,2
0,2022,,NaN
1,2022,,NaN
2,2023,,NaN


In [30]:
df

,v1,v2,col,v3,v4
0,aaa,2022/01,1,AAA,aaa
1,abc,2022/02,2,ABC,abc
2,ccc,2023,3,CCC,ccc
